# Data Cleaning

In deze notebook gaan we werken met een dataset die de gebeurtenissen tijdens een reeks wedstrijden over een periode van 4 jaar.
De link naar deze dataset is: https://www.kaggle.com/secareanualin/football-events.
Met deze kan je reeds heel wat informatie vinden over wat er in deze dataset zit. 
In de dataset zitten er drie files:
* events.csv Bevat data over de gebeurtenissen van elke wedstrijd.
* ginf.csv - Bevat metadata en win-kansen van elke wedstrijd.
* dictionary.txt Bevat een tekstuele weergave van de verschillende categorieen

Hieronder staan alle imports die gebruikt worden in de loop van de notebook.

In [3]:
# loading of the dataset
import opendatasets as od
from zipfile import ZipFile
import pandas as pd
import numpy as np

## Inladen dataset

De eerste stap is om de dataset te downloaden vanuit kaggle met de opendatasets library.
Aangezien de dataset meerdere bestanden bevat kan het zijn dat het wordt gedownload als zip (controleer dit). 
Unzip daarom eerst het bestand indien nodig.
Laad daarna de twee csv bestanden in in een dataframe.
De code voor deze stappen is:

In [ ]:
# download
od.download("https://www.kaggle.com/secareanualin/football-events")

In [4]:
# unzip if necessary en load data in
df_events = pd.read_csv(".\\football-events\\events.csv")
df_ginf = pd.read_csv(".\\football-events\\ginf.csv")

## Exploratory Data Analysis

De read_csv functie heeft een dataframe aangemaakt en alle data van de csv ingeladen in dit object.
Pandas voorziet daarnaast een aantal eenvoudige methodes om heel wat informatie uit de dataset op te halen.
Een overzicht van alle functies die mogelijk zijn met dataframes vind je hier: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html.
We kunnen bijvoorbeeld eenvoudig het aantal rijen gaan opvragen maar ook een aantal rijen gaan bekijken. 
Dit kan bijvoorbeeld met de volgende code:

In [7]:
df_events.info()
df_events.isnull().sum()

id_odsp               0
id_event              0
sort_order            0
time                  0
text                  0
event_type            0
event_type2      726716
side                  0
event_team            0
opponent              0
player            61000
player2          649699
player_in        889294
player_out       889271
shot_place       713550
shot_outcome     712511
is_goal               0
location         473942
bodypart         711824
assist_method         0
situation        711872
fast_break            0
dtype: int64

Probeer nu zelf de volgende zaken op te zoeken:
* Haal willekeurig 20 rijen uit de dataset
* Alle spelers die betrokken zijn geweest bij een event (geen duplicaten)
* Hoeveel goalen zijn er gemaakt in de geregistreerde events. Haal het correcte event type uit de dictionary.txt
* Bestudeer de inhoud van het ginf dataset, besteed vooral aandacht aan de ontbrekende data
* Voeg aan het ginf dataframe twee kolommen toe, 1 voor het aantal events van de thuisploeg en 1 voor het aantal events van de andere ploeg.

In [31]:
# punt 1
df_events.sample(20)

#punt 2
#group by om te groeperen op de kolom player (alle identieke waarden worden gecombineerd)
#size is een aggregatiemethode nodig om het aantal keer te berekenen dat elke speler voorkomt in een event
df_events.groupby("player").size().sort_values()

#punt 3
df_events.is_goal.sum()

#punt 4
events_per_match = df_events.groupby(["id_odsp", "side"]).size().reset_index()
events_per_match_home = events_per_match[events_per_match.side == 1].set_index("id_odsp")
events_per_match_away = events_per_match[events_per_match.side == 2].set_index("id_odsp")
df_ginf["events_home"] = events_per_match_home[0]

df_ginf

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n,events_home
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN,NaN
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN,NaN
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN,NaN
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN,NaN
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10107,xAkY8l6R/,/soccer/italy/serie-a/genoa-crotone-xAkY8l6R/,True,2017-01-22,I1,2017,italy,Genoa,Crotone,2,2,1.97,4.35,8.00,1.95,2.03,2.03,1.86,NaN
10108,xSU9scI9/,/soccer/england/premier-league/chelsea-hull-ci...,True,2017-01-22,E0,2017,england,Chelsea,Hull,2,0,1.19,8.50,20.00,1.54,2.68,2.40,1.66,NaN
10109,xY7uZwOI/,/soccer/france/ligue-1/monaco-lorient-xY7uZwOI/,True,2017-01-22,F1,2017,france,AS Monaco,Lorient,4,0,1.32,6.24,11.50,1.53,3.08,1.80,2.25,NaN
10110,YyeGxMX8/,/soccer/spain/laliga/betis-gijon-YyeGxMX8/,True,2017-01-22,SP1,2017,spain,Real Betis,Sporting Gijon,0,0,1.74,4.07,5.90,2.20,1.89,2.05,1.86,NaN


## Data Cleaning

### Ontbrekende data

In eerste instantie zijn er de volgende mogelijkheden om om te gaan met ontbrekende data (Null or Nan values)
* Negeren
* Verwijder rijen met null waarden
* Verwijder kolommen met null waarden
* Kies een waarde om de NaN waarden in te vullen
* Gebruik duplicaten om de waarde in te vullen

Je kan gebruik maken van de volgende functies:
* isna or notna om aan te geven welke elementen ontbrekende of ingevulde data bevatten 
* dropna drop columns or rows containing missing values  (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
* fillna fill in missing values https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna

Deze functies hebben de volgende resultaten

In [35]:
df_events_copy = df_events.copy()
df_events_copy.isna()

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,...,True,True,True,True,False,True,True,False,True,False
2,False,False,False,False,False,False,True,False,False,False,...,True,True,True,True,False,True,True,False,True,False
3,False,False,False,False,False,False,True,False,False,False,...,True,True,True,True,False,True,True,False,True,False
4,False,False,False,False,False,False,True,False,False,False,...,True,True,True,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,False,False,False,False,False,False,True,False,False,False,...,True,True,True,True,False,False,True,False,True,False
941005,False,False,False,False,False,False,True,False,False,False,...,True,True,True,True,False,True,True,False,True,False
941006,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
941007,False,False,False,False,False,False,True,False,False,False,...,True,True,True,True,False,False,True,False,True,False


We zien dat de manier waarop we dropna uitvoeren een grote impact kan hebben op het resultaat.
In de eerste situatie bekomen we een lege dataset. Dit komt omdat er op elke rij minstens 1 kolom is die een Nan value bevat.

In het tweede voorbeeld, worden alle kolommen verwijderd die minstens 1 Nan value bevat. Op deze manier hebben we nog evenveel elementen maar het aantal kolommen is sterk gereduceerd. Door dit zonder controle te gebruiken kan veel nuttige informatie verwijderd worden. Merk op dat de 4 kolommen met de speler informatie verwijderd zijn. En ook de kolommen die meer informatie bevatten over het type event.

Het merendeel van de verwijderde kolommen bevatten dus nuttige informatie en willen we daarom niet verwijderen.
Om deze reden hebben we een meer manuele aanpak nodig.
De kolommen met informatie over het event (bvb shot_place) zijn nodig en mogen dus blijven staan. 
Nan geeft dan enkel weer dat de waarde niet van toepassing is.
De 4 kolommen met speler informatie kan echter gereduceerd worden tot twee kolommen (1 voor elk team) waarbij player2 niet altijd ingevuld moet zijn.
Dit kan bewezen worden als volgt: 

In [ ]:
df_events_copy.drop("sort_order")

Met bovenstaande resultaten zien we dat in bijna alle gevallen player ofwel player_in ingevuld is maar nooit allebei.
Dit houdt in dat we deze twee kolommen eenvoudig kunnen samenvoegen. 
Hetzelfde geldt voor de kolommen player2 en player_out.
Nog een belangrijke opmerking is dat er een aantal rijen zijn waar er geen informatie over spelers aanwezig is.
Deze rijen kunnen verwijderd worden.
Ook kan de kolom "text" met de tekstuele uitleg van het event verwijderd worden.

Dit doen we hier:

Oefening: Zijn er na het mergen van deze twee kolommen nog gevallen waar player null is maar player2 niet? In het geval dat er nog zo rijen zijn, verplaats de waarde van de player2 kolom naar de player kolom (enkel voor deze rijen). Nadat dit gedaan is hebben we zoveel mogelijk gedaan en kunnen we de overige rijen waarvoor de player feature null is laten vallen. 

### One hot encoding

In deze dataset wordt er vaak een enum gebruikt om een categorie aan te duiden. 
Dit is een compacte manier om een enum in een dataset op te slaan maar dit geeft vaak problemen bij het opbouwen van modellen.
Dit gaan we in meer detail bekijken bij classificatie/clustering.

Om dit op te lossen kunnen gebruik maken van one-hot encoding.
In pandas kan dat door gebruik te maken van de functie pd.get_dummies()

### Data Format Correcting

Buiten het verwijderen of toevoegen van kolommen kan het ook belangrijk zijn om de waarden van individuele elementen te corrigeren/aan te passen. 
Dit kan een heel aantal zaken omvatten zoals het corrigeren van typo's, omzetten van datatypes, etc.
Laten we eerst eens de One-Hot Encoded values omzetten van 0/1 naar true/false.

Daarnaast moeten er ook vaak woorden in tekstuele velden gecorrigeerd of vertaald worden. 
Om hiervan een voorbeeld te geven ga ik in de player en player2 kolommen de naam van "Romelu Lukaku" hernoemen naar "Big Rom".
De volgende code voert dit uit door gebruik te maken van de [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html) functie:

Een andere grote bron van data formaten dat aangepast moet worden komt voort uit kolommen die datums bevatten. Deze worden standaard ingelezen als een tekst veldje dus moet je deze zelf gaan omzetten. Dit kan je zien in de ginf dataset waar de "date" kolom van het type object is. Dit is in numpy de indicatie dat het een tekstveld is.

In de dataframe zien we dat de datums erin staan als "%Y-%m-%d" (jaar als 4 cijfers, rest als twee). 
Voor de notatie van andere dataformaten vind je [hier](http://strftime.org/) meer informatie.
Met behulp van de [to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) functie kan je deze kolom omzetten naar datums.
Zoek na deze omzetting eens het datatype op van deze kolom.
Je ziet dat het nu geen object meer is maar een correcte datum.

Merk op dat deze manier van aanpakken enkel werkt wanneer de hele tabel hetzelfde dataformaat heeft.
Indien er meerdere dataformaten gebruikt zijn (bijvoorbeeld door manuele input), dan kan je het nog steeds doen maar dan moet panda een inschatting maken van het gewenste formaat. Dit kan door het argument "infer_datetime_format=True".
Hoewel dit mogelijk is zijn er twee redenen waarom we dit niet altijd gebruiken, namelijk
* Het duurt veel langer om de tabel om te zetten.
* Het gaat niet altijd het correcte resultaat geven bij speciale formaten

Eenmaal de kolom van het type datetime is kan je heel gemakkelijk deze kolommen vergelijken en er zaken gaan uithalen zoals de maand waarin de wedstrijd plaatsvond. 

In [142]:
df_ginf["datum"].dt.month

0        8
1        8
2        8
3        8
4        8
        ..
10107    1
10108    1
10109    1
10110    1
10111    1
Name: datum, Length: 10112, dtype: int64

### Data shuffling

Een mogelijke manier om de privacy van je klanten te beschermen die we aangehaald hebben is data shuffling.
Bij deze methode is het de bedoeling dat je de namen in een dataset op een willekeurige manier door elkaar haalt.
Dit kan gedaan worden door gebruik te maken van de permutation methode van numpy  (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) op de volgende manier:

## Oefeningen

Om meer vertrouwd te worden met het werken met pandas kan je de volgende zaken zelf proberen.
Indien het niet lukt, mag je het me gerust vragen of het proberen zelf op te zoeken naar wat de oorzaak is.

Bewerkingen op de events.csv dataset:
* Controleer de kolommen "id_event", "id_odsp" en "sort_order". Is er 1 van deze drie overbodig? Verwijder de kolom indien dit zo is.
* Hernoem "Hamburg SV" naar "Hamburg FC". Let op, doe dit voor alle kolommen waar het kan staan.

Bewerkingen op de ginf.csv dataset:
* Maak One Hot Encoding voor de kolom "Country"
* Voeg een kolom toe dat drie waarden kan hebben: Home team wins, Home team loses, draw
* Maak nu ook een One Hot Encoding voor deze nieuwe kolom
* Drop de kolom met de urls voor de ploegen
* Bewaar je aangepaste dataset in "wedstrijden.csv" (Gebruik de functie to_save: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)